In [28]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import push_notebook, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, FactorRange, Whisker
from bokeh.models.glyphs import VArea
from bokeh.models.tools import HoverTool
import bokeh.palettes
from bokeh.transform import factor_cmap
import math 
import pandas as pd

df = pd.read_csv('Results/ab_final3_35.csv')
print(df)


        runtime image_name ressource_limit  number_of_targets  targetId  \
0       default   go-httpd         default                  1         1   
1       default   go-httpd         default                  1         1   
2       default   go-httpd         default                  1         1   
3       default   go-httpd         default                  1         1   
4       default   go-httpd         default                  1         1   
...         ...        ...             ...                ...       ...   
115145    runsc     tomcat             min                  8         4   
115146    runsc     tomcat             min                  8         5   
115147    runsc     tomcat             min                  8         6   
115148    runsc     tomcat             min                  8         7   
115149    runsc     tomcat             min                  8         8   

        repeat  completed_requests  failed_requests  requests_per_second  \
0            1         

In [29]:
# Exclude gvisor with httpd
filterd = df[(df.runtime != 'runsc')]
#print(filterd)
df_gvisor = df[(df.runtime == 'runsc')]
#print(df_gvisor)
df_gvisor = df_gvisor[(df_gvisor.image_name != 'httpd')]
#print(df_gvisor)

frames = [filterd, df_gvisor]
filterd = pd.concat(frames)
print(filterd)
filterd = filterd[(filterd.image_name != 'node-express')]
print(filterd)

        runtime image_name ressource_limit  number_of_targets  targetId  \
0       default   go-httpd         default                  1         1   
1       default   go-httpd         default                  1         1   
2       default   go-httpd         default                  1         1   
3       default   go-httpd         default                  1         1   
4       default   go-httpd         default                  1         1   
...         ...        ...             ...                ...       ...   
115145    runsc     tomcat             min                  8         4   
115146    runsc     tomcat             min                  8         5   
115147    runsc     tomcat             min                  8         6   
115148    runsc     tomcat             min                  8         7   
115149    runsc     tomcat             min                  8         8   

        repeat  completed_requests  failed_requests  requests_per_second  \
0            1         

In [30]:
# Remove -1 Values
filterd = filterd[(filterd.requests_per_second != -1)]
print(filterd)

        runtime image_name ressource_limit  number_of_targets  targetId  \
0       default   go-httpd         default                  1         1   
1       default   go-httpd         default                  1         1   
2       default   go-httpd         default                  1         1   
3       default   go-httpd         default                  1         1   
4       default   go-httpd         default                  1         1   
...         ...        ...             ...                ...       ...   
115145    runsc     tomcat             min                  8         4   
115146    runsc     tomcat             min                  8         5   
115147    runsc     tomcat             min                  8         6   
115148    runsc     tomcat             min                  8         7   
115149    runsc     tomcat             min                  8         8   

        repeat  completed_requests  failed_requests  requests_per_second  \
0            1         

In [31]:
grouped = filterd.groupby(['runtime', 'image_name', 'ressource_limit', 'number_of_targets', 'repeat'])['requests_per_second'].mean().reset_index(name='mean_requests_per_second')
print(grouped)

       runtime image_name ressource_limit  number_of_targets  repeat  \
0      default   go-httpd         default                  1       1   
1      default   go-httpd         default                  1       2   
2      default   go-httpd         default                  1       3   
3      default   go-httpd         default                  1       4   
4      default   go-httpd         default                  1       5   
...        ...        ...             ...                ...     ...   
14466    runsc     tomcat             min                 21      31   
14467    runsc     tomcat             min                 21      32   
14468    runsc     tomcat             min                 21      33   
14469    runsc     tomcat             min                 21      34   
14470    runsc     tomcat             min                 21      35   

       mean_requests_per_second  
0                  20727.850000  
1                  21159.900000  
2                  21092.160000  

In [32]:
grouped['sum_requests_per_second'] = grouped['mean_requests_per_second'] * grouped['number_of_targets']
print(grouped)

       runtime image_name ressource_limit  number_of_targets  repeat  \
0      default   go-httpd         default                  1       1   
1      default   go-httpd         default                  1       2   
2      default   go-httpd         default                  1       3   
3      default   go-httpd         default                  1       4   
4      default   go-httpd         default                  1       5   
...        ...        ...             ...                ...     ...   
14466    runsc     tomcat             min                 21      31   
14467    runsc     tomcat             min                 21      32   
14468    runsc     tomcat             min                 21      33   
14469    runsc     tomcat             min                 21      34   
14470    runsc     tomcat             min                 21      35   

       mean_requests_per_second  sum_requests_per_second  
0                  20727.850000                 20727.85  
1                

In [33]:
# Fehlerrechnung

std = grouped.groupby(['runtime', 'image_name', 'ressource_limit', 'number_of_targets'])['sum_requests_per_second'].std().reset_index(name='std')
print(std)

mean = grouped.groupby(['runtime', 'image_name', 'ressource_limit', 'number_of_targets'])['sum_requests_per_second'].mean().reset_index(name='mean')
print(mean)

std['mean'] = mean[['mean']]
grouped_std_mean = std
print(grouped_std_mean)

# 95% Vertrauensintervall
grouped_std_mean['abweichung_in_prozent'] = grouped_std_mean['std']  / math.sqrt(35) * 1.6909 * 100 / grouped_std_mean['mean']
grouped_std_mean['konf_intervall_unten'] = grouped_std_mean['mean'] - grouped_std_mean['std']  / math.sqrt(35) * 1.6909
grouped_std_mean['konf_intervall_oben'] = grouped_std_mean['mean'] + grouped_std_mean['std']  / math.sqrt(35) * 1.6909

grouped_std_mean = grouped_std_mean.round(2)
print(grouped_std_mean)

#summe_delta_s_quadrat = filterd['delta_s_quadrat'].sum()
#standartabweichung = math.sqrt(summe_delta_s_quadrat/199)
#print('standartabweichung: ' + str(standartabweichung))

# Formel t Verteilung: http://www.statistics4u.info/fundstat_germ/cc_confidence_interval.html
# T Verteilungs tabelle: https://www.staff.uni-mainz.de/klenke/vorlesungen/vorl_ws19-2/tabellenwerk.pdf
# Nachschauen:
# n -1 = 39 <- richtig?
# a = 0,005 bei 99,5% Wahrscheinlichkeit
# Werte 38-> 2,712 40-> 2,704 ~ 39 -> 2,7105
# neu: 39 -> 2.7079
# Formel:
# Erwartungswert +- Standartabweichung / sqrt(40) * 2,7105

     runtime image_name ressource_limit  number_of_targets         std
0    default   go-httpd         default                  1  258.653857
1    default   go-httpd         default                  2   87.969456
2    default   go-httpd         default                  3   91.723980
3    default   go-httpd         default                  5  114.177105
4    default   go-httpd         default                  8  142.167386
..       ...        ...             ...                ...         ...
409    runsc     tomcat             min                  3   11.919818
410    runsc     tomcat             min                  5  150.218744
411    runsc     tomcat             min                  8   59.832636
412    runsc     tomcat             min                 13   54.626987
413    runsc     tomcat             min                 21   43.925822

[414 rows x 5 columns]
     runtime image_name ressource_limit  number_of_targets          mean
0    default   go-httpd         default            

In [39]:
######
# Leistung indivduell
####

source = ColumnDataSource(pd.DataFrame(grouped_std_mean))
#print(source.data)

image_names = list( dict.fromkeys(source.data['image_name'].tolist()))
#print(image_names)

runtimes = list( dict.fromkeys(source.data['runtime'].tolist()))
#print(runtimes)

ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
#print(ressource_limits)

color_dict = {'default': 0, 'kata': 2, 'katafc': 3, 'runsc': 4, 'runnc': 5} 
legend_dict = {'default': 'Docker (runc)', 'kata': 'Kata', 'katafc': 'Kata FC', 'runsc': 'gVisor (runsc)', 'runnc': 'Nabla (runnc)'} 
dash_dict = {'default': 'solid', 'max': 'dashed', 'min': 'dotted'}
for image_name in image_names:
    p = figure(x_axis_type='linear', width=1500, height=1100)
    filterd=grouped_std_mean[grouped_std_mean['image_name'] == image_name]
    
    source = ColumnDataSource(pd.DataFrame(filterd))
    #print(source.data)

    runtimes = list( dict.fromkeys(source.data['runtime'].tolist()))
    #print(runtimes)

    ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
    #print(ressource_limits)
    
    colour_counter = 0
    for runtime in runtimes:
        filterd_rnt=filterd[filterd['runtime'] == runtime]
        
        source = ColumnDataSource(pd.DataFrame(filterd_rnt))
        #print(source.data)

        ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
        #print(ressource_limits)
        
        for limit in ressource_limits:
            filterd_limit=filterd_rnt[filterd_rnt['ressource_limit'] == limit]
        
            source = ColumnDataSource(pd.DataFrame(filterd_limit))
            #print(source.data)
            #glyph = VArea(x='number_of_targets', y1='min_requests_per_second', y2='max_requests_per_second', fill_color=bokeh.palettes.d3['Category20'][12][1], fill_alpha=0.3)
            #p.add_glyph(default_source, glyph)
            p.line(x='number_of_targets', y='mean', line_width=2, line_dash=dash_dict[limit], source=source, color=bokeh.palettes.d3['Category10'][6][color_dict[runtime]], legend_label=legend_dict[runtime] + ' - ' + limit)
            
            
            p.add_layout(
              Whisker(source=source, base='number_of_targets', upper="konf_intervall_oben", lower="konf_intervall_unten", level="overlay")
            )
        print(bokeh.palettes.d3['Category10'][6][color_dict[runtime]])
        colour_counter = colour_counter + 1
    
    p.title.text ='Anfragen pro Sekunde nach Anzahl der parallelen Container für ' + image_name
    p.xaxis.axis_label = 'Anzahl an parallelen Containern'
    p.yaxis.axis_label = 'Anfragen pro Sekunde'
    p.x_range.start = 1
    p.xaxis.ticker = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
    p.legend.click_policy="hide"
    p.legend.location = "top_left"
    p.legend.background_fill_alpha = 0.5
    p.title.text_font_size = '16pt'
    p.legend.label_text_font_size = '14pt'
    p.xaxis.axis_label_text_font_size = "15pt"
    p.xaxis.major_label_text_font_size = "13pt"

    p.yaxis.axis_label_text_font_size = "15pt"
    p.yaxis.major_label_text_font_size = "13pt"
    
    output_file("requests_pro_sec_anzahl_container_" + image_name + ".html")
    output_notebook()
    show(p)

#1f77b4
#2ca02c
#d62728
#8c564b
#9467bd


Loading BokehJS ...

#1f77b4
#2ca02c
#d62728


Loading BokehJS ...

#1f77b4
#2ca02c
#d62728
#9467bd


Loading BokehJS ...

#1f77b4
#2ca02c
#d62728
#8c564b
#9467bd


Loading BokehJS ...

#1f77b4
#2ca02c
#9467bd


Loading BokehJS ...

In [160]:
######
# Leistung indivduell
####

source = ColumnDataSource(pd.DataFrame(grouped_std_mean))
#print(source.data)

image_names = list( dict.fromkeys(source.data['image_name'].tolist()))
#print(image_names)

number_of_targets = list( dict.fromkeys(source.data['number_of_targets'].tolist()))
#print(image_names)

runtimes = list( dict.fromkeys(source.data['runtime'].tolist()))
#print(runtimes)

ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
#print(ressource_limits)

#x = [ (image_name, ressource_limit, runtime) for image_name in image_names for ressource_limit in ressource_limits for runtime in runtimes ]
#print(x)

for image_name in image_names:
    for targets in number_of_targets:
        
        
        filterd=grouped_std_mean[grouped_std_mean['image_name'] == image_name]
        filterd=filterd[filterd['number_of_targets'] == targets]

        source = ColumnDataSource(pd.DataFrame(filterd))
        #print(source.data)

        runtimes = list(dict.fromkeys(source.data['runtime'].tolist()))
        #print(runtimes)

        ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
        #print(ressource_limits)

        x = [ (ressource_limit, runtime) for ressource_limit in ressource_limits for runtime in runtimes ]
        print(x)

        source = ColumnDataSource(data=dict(x=x, mean=source.data['mean'].tolist(), oben=source.data['konf_intervall_oben'].tolist(), unten=source.data['konf_intervall_unten'].tolist()))
        print(source.data)
        p = figure(x_range=FactorRange(*x), width=1000)

        # use the palette to colormap based on the the x[1:2] values
        #fill_color=factor_cmap('x', palette=bokeh.palettes.d3['Category10'][5], factors=runtime, start=1, end=2)

        #p.vbar(x='x', top='mean', source=source, width=0.70, color=fill_color)
        p.vbar(x='x', top='mean', source=source, width=0.70)

        p.add_layout(
          Whisker(source=source, base='x', upper="oben", lower="unten", level="overlay")
        )


        p.title.text ='Startzeit für Image ' + image_name + ' mit der Skalierung' + str(targets)
        p.xaxis.axis_label = 'Ressourcelimit & Runtime'
        p.yaxis.axis_label = 'Startzeit in Sekunden'

        p.xaxis.axis_label_text_font_size = '11pt'
        p.yaxis.axis_label_text_font_size = '11pt'
       


        output_notebook()

        show(p)

[('default', 'default'), ('default', 'kata'), ('default', 'katafc'), ('default', 'runnc'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'katafc'), ('max', 'runnc'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'katafc'), ('min', 'runnc'), ('min', 'runsc')]
{'x': [('default', 'default'), ('default', 'kata'), ('default', 'katafc'), ('default', 'runnc'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'katafc'), ('max', 'runnc'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'katafc'), ('min', 'runnc'), ('min', 'runsc')], 'mean': [21012.2, 21816.45, 2691.59, 16436.86, 19276.83, 18388.69, 10020.57, 9961.83, 10076.52, 7608.3, 7452.65, 7547.29, 7598.78, 7731.29, 491.33], 'oben': [21086.13, 21874.7, 2731.0, 16506.24, 19334.06, 18638.51, 10079.3, 10064.9, 10135.2, 7714.29, 7612.32, 7670.11, 7612.58, 7743.47, 492.46], 'unten': [20938.28, 21758.2, 2652.19, 16367.47, 19219.59, 18138.87, 9961.83, 9858.75, 10017.84, 7502.3,

/usr/local/lib/python3.7/dist-packages/bokeh/models/plots.py:767: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


Loading BokehJS ...

[('default', 'default'), ('default', 'kata'), ('default', 'katafc'), ('default', 'runnc'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'katafc'), ('max', 'runnc'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'katafc'), ('min', 'runnc'), ('min', 'runsc')]
{'x': [('default', 'default'), ('default', 'kata'), ('default', 'katafc'), ('default', 'runnc'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'katafc'), ('max', 'runnc'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'katafc'), ('min', 'runnc'), ('min', 'runsc')], 'mean': [26793.03, 27374.09, 17759.8, 30137.01, 28063.36, 26437.41, 17376.48, 17437.72, 17381.52, 15292.82, 15211.81, 15244.59, 8545.24, 8670.54, 1094.93], 'oben': [26818.17, 27413.68, 18011.17, 30304.47, 28129.95, 26503.79, 17422.54, 17479.35, 17424.64, 15359.94, 15235.51, 15347.63, 8564.41, 8683.83, 1096.47], 'unten': [26767.88, 27334.51, 17508.43, 29969.56, 27996.77, 26371.03, 17330.42, 17396.

Loading BokehJS ...

...

Loading BokehJS ...

[('default', 'default'), ('default', 'kata'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'runsc')]
{'x': [('default', 'default'), ('default', 'kata'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'runsc')], 'mean': [22487.95, 22341.32, 22001.87, 20165.58, 23348.12, 24802.69, 5266.81, 5214.04, 4232.27], 'oben': [22581.62, 22438.46, 22067.2, 20331.3, 23482.67, 24996.85, 5287.22, 5229.04, 4247.88], 'unten': [22394.29, 22244.18, 21936.54, 19999.87, 23213.56, 24608.54, 5246.39, 5199.03, 4216.65]}


Loading BokehJS ...

[('default', 'default'), ('default', 'kata'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'runsc')]
{'x': [('default', 'default'), ('default', 'kata'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'runsc')], 'mean': [23592.54, 23013.67, 24248.15, 20505.74, 17395.57, 23152.98, 5004.41, 4865.39, 4081.74], 'oben': [23745.58, 23145.82, 24353.09, 20640.24, 17511.04, 23353.91, 5027.68, 4888.05, 4094.29], 'unten': [23439.5, 22881.51, 24143.21, 20371.24, 17280.09, 22952.06, 4981.14, 4842.73, 4069.19]}


Loading BokehJS ...

In [150]:
print(grouped_std_mean.to_string())


     runtime      image_name ressource_limit  number_of_targets      std      mean  abweichung_in_prozent  konf_intervall_unten  konf_intervall_oben  docker_abweichung
0    default        go-httpd         default                  1   258.65  21012.20                   0.35              20938.28             21086.13             100.00
1    default        go-httpd         default                  2    87.97  26793.03                   0.09              26767.88             26818.17             100.00
2    default        go-httpd         default                  3    91.72  26138.46                   0.10              26112.24             26164.67             100.00
3    default        go-httpd         default                  5   114.18  25828.78                   0.13              25796.14             25861.41             100.00
4    default        go-httpd         default                  8   142.17  27089.24                   0.15              27048.60             27129.87            

In [154]:
grouped_std_mean['docker_abweichung'] = 0

#print(grouped_std_mean.loc[ 273 , : ])

for index, row in grouped_std_mean.iterrows():
    docker_mean = grouped_std_mean.loc[(grouped_std_mean['runtime'] == 'default') & (grouped_std_mean['number_of_targets'] == row['number_of_targets']) & (grouped_std_mean['image_name'] == row['image_name']) & (grouped_std_mean['ressource_limit'] == row['ressource_limit']), ['mean']].reset_index().get_value(0, 'mean')
    #print(index)
    #print(row['mean'] * 100 / docker_mean)
    grouped_std_mean.loc[index, 'docker_abweichung'] = row['mean'] * 100 / docker_mean

grouped_std_mean = grouped_std_mean.round(2)

print(grouped_std_mean.to_string())

export_csv = grouped_std_mean.to_csv (r'Results/ab_final_35_Gewichtung_Docker.csv', index = None, header=True, decimal=',')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


     runtime      image_name ressource_limit  number_of_targets      std      mean  abweichung_in_prozent  konf_intervall_unten  konf_intervall_oben  docker_abweichung
0    default        go-httpd         default                  1   258.65  21012.20                   0.35              20938.28             21086.13             100.00
1    default        go-httpd         default                  2    87.97  26793.03                   0.09              26767.88             26818.17             100.00
2    default        go-httpd         default                  3    91.72  26138.46                   0.10              26112.24             26164.67             100.00
3    default        go-httpd         default                  5   114.18  25828.78                   0.13              25796.14             25861.41             100.00
4    default        go-httpd         default                  8   142.17  27089.24                   0.15              27048.60             27129.87            